## CSTR Non-linear Example

This shows plain simulation of coupled ordinary differential equations.

In [96]:
from pyomo.dae import *
kA  = 0.5   
kB  = 0.1   
cA0 = 2.0   

m = ConcreteModel()

m.t = ContinuousSet(bounds=(0, 5))

m.cA = Var(m.t, domain=NonNegativeReals)
m.cB = Var(m.t, domain=NonNegativeReals)
m.cC = Var(m.t, domain=NonNegativeReals)

m.dcA = DerivativeVar(m.cA)
m.dcB = DerivativeVar(m.cB)
m.dcC = DerivativeVar(m.cC)

m.odeA = Constraint(m.t,
    rule=lambda m, t: m.dcA[t] == (-kA*m.cA[t]) if t > 0 else Constraint.Skip)
m.odeB = Constraint(m.t,
    rule=lambda m, t: m.dcB[t] == (kA*m.cA[t] - kB*m.cB[t]) if t > 0 else Constraint.Skip)
m.odeC = Constraint(m.t,
    rule=lambda m, t: m.dcC[t] == (kB*m.cB[t]) if t > 0 else Constraint.Skip)

m.ic = ConstraintList()
m.ic.add(m.cA[0]  == cAf)
m.ic.add(m.cB[0]  == 0)
m.ic.add(m.cC[0]  == 0)

TransformationFactory('dae.collocation').apply_to(m)
SolverFactory('ipopt').solve(m)
print('Outlet c_A:', m.cA[5]())
print('Outlet c_B:', m.cB[5]())
print('Outlet c_C:', m.cC[5]())

Outlet c_A: 0.16417005073230254
Outlet c_B: 1.311114085898802
Outlet c_C: 0.5247158633688539


Here we transform into non-dimensional variables, then optimize with respect to time.

In [97]:
kA  = 0.5   
kB  = 0.1   
cA0 = 2.0   

m = ConcreteModel()

m.tau = ContinuousSet(bounds=(0, 1))

m.tf = Var(domain=NonNegativeReals)
m.cA = Var(m.tau, domain=NonNegativeReals)
m.cB = Var(m.tau, domain=NonNegativeReals)
m.cC = Var(m.tau, domain=NonNegativeReals)

m.dcA = DerivativeVar(m.cA)
m.dcB = DerivativeVar(m.cB)
m.dcC = DerivativeVar(m.cC)

m.odeA = Constraint(m.tau,
    rule=lambda m, tau: m.dcA[tau] == m.tf*(-kA*m.cA[tau]) if tau > 0 else Constraint.Skip)
m.odeB = Constraint(m.tau,
    rule=lambda m, tau: m.dcB[tau] == m.tf*(kA*m.cA[tau] - kB*m.cB[tau]) if tau > 0 else Constraint.Skip)
m.odeC = Constraint(m.tau,
    rule=lambda m, tau: m.dcC[tau] == m.tf*(kB*m.cB[tau]) if tau > 0 else Constraint.Skip)

m.ic = ConstraintList()
m.ic.add(m.cA[0]  == cAf)
m.ic.add(m.cB[0]  == 0)
m.ic.add(m.cC[0]  == 0)

m.obj = Objective(expr=m.cB[1]*10+m.cC[1]*3, sense=maximize)

TransformationFactory('dae.collocation').apply_to(m)
SolverFactory('ipopt').solve(m)
print('Maximum Profit is $', 10*m.cB[1]()+3*m.cC[1](), 'with residence time:', m.tf(), 'minutes.')
print('Outlet c_A:', m.cA[1]())
print('Outlet c_B:', m.cB[1]())
print('Outlet c_C:', m.cC[1]())

Maximum Profit is $ 14.697171916092724 with residence time: 4.7605927456556 minutes.
Outlet c_A: 0.18504634917716825
Outlet c_B: 1.321758709089196
Outlet c_C: 0.4931949417335876
